In [1]:
%%capture
!uv pip install polars

In [2]:
import polars as pl

In [4]:
df = pl.read_csv("../data/chess_games_2025-01-15.csv", null_values=["None"])

In [5]:
df.head()

Date,Result,WhiteElo,BlackElo,PGN
str,str,i64,i64,str
"""2000.03.14""","""1-0""",2851,null,"""1.d4 d5 2.c4 e6 3.Nc3 Nf6 4.cx…"
"""2000.03.14""","""1-0""",2851,null,"""1.e4 d5 2.exd5 Qxd5 3.Nc3 Qa5 …"
"""1999.11.20""","""1-0""",2851,null,"""1.e4 e5 2.Nf3 Nc6 3.Bc4 Bc5 4.…"
"""1999.11.20""","""1-0""",2851,null,"""1.e4 d5 2.exd5 Qxd5 3.Nc3 Qa5 …"
"""2000.02.20""","""1/2-1/2""",2851,2633,"""1.e4 e5 2.Nf3 Nc6 3.Bb5 a6 4.B…"


In [89]:
test_game = df.select("PGN")[0].item()

In [90]:
tokens = test_game.encode("utf-8")
token_ids = list(map(int, tokens))

print(test_game)
print("length: ", len(test_game))
print(token_ids)
print("length:", len(token_ids))

1.d4 d5 2.c4 e6 3.Nc3 Nf6 4.cxd5 exd5 5.Bg5 Be7 6.e3 Ne4 7.Bxe7 Nxc3 8.Bxd8 Nxd1 9.Bxc7 Nxb2 10.Rb1 Nc4 11.Bxc4 dxc4 12.Ne2 O-O 13.Nc3 b6 14.d5 Na6 15.Bd6 Rd8 16.Ba3 Bb7 17.e4 f6 18.Ke2 Nc7 19.Rhd1 Ba6 20.Ke3 Kf7 21.g4 g5 22.h4 h6 23.Rh1 Re8 24.f3 Bb7 25.hxg5 fxg5 26.d6 Nd5+ 27.Nxd5 Bxd5 28.Rxh6 c3 29.d7 Re6 30.Rh7+ Kg8 31.Rbh1 Bc6 32.Rh8+ Kf7 33.Rxa8 Bxd7 34.Rh7+ 
length:  367
[49, 46, 100, 52, 32, 100, 53, 32, 50, 46, 99, 52, 32, 101, 54, 32, 51, 46, 78, 99, 51, 32, 78, 102, 54, 32, 52, 46, 99, 120, 100, 53, 32, 101, 120, 100, 53, 32, 53, 46, 66, 103, 53, 32, 66, 101, 55, 32, 54, 46, 101, 51, 32, 78, 101, 52, 32, 55, 46, 66, 120, 101, 55, 32, 78, 120, 99, 51, 32, 56, 46, 66, 120, 100, 56, 32, 78, 120, 100, 49, 32, 57, 46, 66, 120, 99, 55, 32, 78, 120, 98, 50, 32, 49, 48, 46, 82, 98, 49, 32, 78, 99, 52, 32, 49, 49, 46, 66, 120, 99, 52, 32, 100, 120, 99, 52, 32, 49, 50, 46, 78, 101, 50, 32, 79, 45, 79, 32, 49, 51, 46, 78, 99, 51, 32, 98, 54, 32, 49, 52, 46, 100, 53, 32, 78, 97, 54, 32,

In [91]:
# get the most common pair of tokens
def find_token_pairs(ids):
    counts = {}

    # get pairs of tokens
    for pair in zip(ids, ids[1:]):
        if pair not in counts:
            counts[pair] = 1
        else:
            counts[pair] += 1

    return counts

In [92]:
data_start = "<|startgame|>"
data_end = "<|endgame|>"

data_start_tokens = data_start.encode("utf-8")
data_end_tokens = data_end.encode("utf-8")

data_start_token_ids = list(map(int, data_start_tokens))
data_end_token_ids = list(map(int, data_end_tokens))

print(data_start_token_ids)
print(data_end_token_ids)

tokenizer_special_tokens = {
    "startgame": data_start,
    "endgame": data_end,
}

[60, 124, 115, 116, 97, 114, 116, 103, 97, 109, 101, 124, 62]
[60, 124, 101, 110, 100, 103, 97, 109, 101, 124, 62]


In [93]:
token_pairs = find_token_pairs(token_ids)

# print(tokenPairs)
print(sorted(((v, k) for k, v in token_pairs.items()), reverse=True))

[(12, (54, 32)), (11, (32, 50)), (10, (55, 32)), (10, (53, 32)), (10, (32, 49)), (9, (52, 32)), (9, (46, 82)), (9, (32, 78)), (8, (51, 32)), (7, (120, 100)), (7, (100, 53)), (7, (46, 66)), (7, (32, 66)), (6, (66, 120)), (6, (32, 51)), (5, (82, 104)), (5, (56, 32)), (5, (49, 32)), (4, (120, 99)), (4, (103, 53)), (4, (99, 52)), (4, (99, 51)), (4, (78, 120)), (4, (78, 99)), (4, (52, 46)), (4, (51, 46)), (4, (50, 46)), (4, (49, 46)), (4, (46, 100)), (4, (46, 78)), (4, (43, 32)), (3, (57, 46)), (3, (56, 46)), (3, (55, 46)), (3, (54, 46)), (3, (53, 46)), (3, (50, 32)), (3, (48, 46)), (3, (32, 82)), (3, (32, 75)), (2, (120, 103)), (2, (104, 55)), (2, (104, 54)), (2, (104, 49)), (2, (102, 55)), (2, (102, 54)), (2, (101, 55)), (2, (101, 54)), (2, (101, 52)), (2, (101, 51)), (2, (101, 50)), (2, (100, 56)), (2, (100, 55)), (2, (100, 54)), (2, (100, 49)), (2, (99, 55)), (2, (98, 55)), (2, (97, 54)), (2, (82, 120)), (2, (82, 101)), (2, (82, 98)), (2, (78, 101)), (2, (75, 102)), (2, (75, 101)), (2, 

In [94]:
top_pair = max(token_pairs, key=token_pairs.get)
top_pair

(54, 32)

In [95]:
def merge_pairs(ids, pair, idx):
    new_ids = []

    i = 0

    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            new_ids.append(idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1

        return new_ids

In [96]:
tokens2 = merge_pairs(token_ids, top_pair, 256)

print(tokens2)
print("length:", len(tokens2))

[49]
length: 1


In [103]:
sample = df.select("PGN").sample(n=20)

In [104]:
text = ""

for game in sample.iter_rows():
    if game[0]:
        text += game[0].strip() + "\n"

In [105]:
text_tokens = text.encode("utf-8")
text_tokens = list(map(int, text_tokens))

In [108]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts


def merge_pairs(ids, pair, idx):
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            new_ids.append(idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids


vocab_size = 350
number_of_merges = vocab_size - 256
ids = list(text_tokens)

merges = {}

for i in range(number_of_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i
    print(f"merge {pair} into a new token {idx}")
    ids = merge_pairs(ids, pair, idx)
    merges[pair] = idx

merge (52, 32) into a new token 256
merge (53, 32) into a new token 257
merge (51, 32) into a new token 258
merge (54, 32) into a new token 259
merge (46, 78) into a new token 260
merge (50, 32) into a new token 261
merge (55, 32) into a new token 262
merge (46, 66) into a new token 263
merge (46, 82) into a new token 264
merge (46, 81) into a new token 265
merge (49, 32) into a new token 266
merge (56, 32) into a new token 267
merge (43, 32) into a new token 268
merge (102, 258) into a new token 269
merge (120, 100) into a new token 270
merge (120, 101) into a new token 271
merge (102, 259) into a new token 272
merge (257, 50) into a new token 273
merge (46, 75) into a new token 274
merge (79, 45) into a new token 275
merge (257, 49) into a new token 276
merge (275, 79) into a new token 277
merge (277, 32) into a new token 278
merge (120, 103) into a new token 279
merge (99, 259) into a new token 280
merge (99, 256) into a new token 281
merge (49, 46) into a new token 282
merge (99, 2

In [109]:
print("text length:", len(text))
print("tokens length:", len(text_tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(text_tokens) / len(ids):.2f}")

text length: 7463
tokens length: 7463
ids length: 3896
compression ratio: 1.92


In [115]:
vocab = {idx: bytes([idx]) for idx in range(256)}

for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]


def decode(ids: list[int]) -> str:
    tokens = b"".join(vocab[idx] for idx in ids)

    text = tokens.decode("utf-8", errors="replace")

    return text


decode(ids)

'1.d4 d5 2.Nf3 Bf5 3.g3 c6 4.Bg2 e6 5.O-O Nf6 6.b3 Bd6 7.Bb2 Nbd7 8.Nbd2 O-O 9.Nh4 Re8 10.c4 Rc8 11.Rc1 Bg4 12.f3 Bh5 13.f4 Ng4 14.Rc3 g5 15.fxg5 Qxg5 16.Ndf3 Qe7 17.Bc1 Ndf6 18.h3 Ne4 19.hxg4 Nxc3 20.Qd3 Bxg4 21.Qxc3 Bxg3 22.Bg5 f6 23.Bd2 Bxh4 24.Nxh4 Bxe2 25.Rf2 Bh5 26.Qh3 Kh8 27.Bf3 Rg8+ 28.Rg2 Rxg2+ 29.Kxg2 Rg8+ 30.Kf2 Bxf3 31.Nxf3 f5 32.Bf4 Rg6 33.Ne5 Rg7 34.Bh6 Rg8 35.Qh5 Qe8 36.Nf7+ Qxf7\n1.d4 d5 2.Nf3 Nf6 3.g3 c6 4.Bg2 Bf5 5.Nbd2 h6 6.O-O e6 7.b3 Bd6 8.Bb2 O-O 9.Ne5 Nbd7 10.f3 Qc7 11.Nxd7 Nxd7 12.e4 dxe4 13.fxe4 Bg6 14.e5 Be7 15.c3 c5 16.Nc4 Rad8 17.Qe2 Nb6 18.Ne3 Rd7 19.Rad1 Rc8 20.h4 Re8 21.Kh2 Bf8 22.a4 Red8 23.a5 Nc8 24.a6 bxa6 25.Qxa6 Qb8 26.Qa2 Ne7 27.Rf2 Nf5 28.Nxf5 Bxf5 29.Rfd2 Bg4 30.Re1 Qb6 31.Qa4 cxd4 32.Rxd4 Rxd4 33.cxd4 Rc8 34.d5 Qf2 35.Rf1 Qe2 36.Bd4 exd5 37.Qxa7 Be6 38.Rf2 Qd1 39.Qa4 Rc4 40.Rf1 Qc2 41.Qa7 Qxb3 42.Rf2 Qd1 43.Bb2 Bc5 44.Qb8+ Kh7 45.Rf1 Qd2 46.Bc1 Rxc1\n1.e4 e5 2.Nf3 Nc6 3.Bc4 Bc5 4.O-O Nf6 5.d3 d6 6.Ng5 O-O 7.Nf3 Bg4 8.h3 Bh5 9.Bg5 h6 10.Bxf6 Qxf6 

In [116]:
def encode(text: str) -> list[int]:
    tokens = list(text.encode("utf-8"))

    while len(tokens) >= 2:
        stats = get_stats(tokens)

        pair = min(stats, key=lambda p: merges.get(p, float("inf")))

        if pair not in merges:
            break

        idx = merges[pair]

        tokens = merge_pairs(tokens, pair, idx)

    return tokens


print(encode("1.d4 d5 2.Nf3 Bf5"))

[282, 291, 100, 273, 303, 66, 102, 53]


In [117]:
print(decode(encode("1.d4 d5 2.Nf3 Bf5")))

1.d4 d5 2.Nf3 Bf5


In [143]:
new_sample = df.select("PGN").sample(n=1)[0].item().strip()

print(new_sample == decode(encode(new_sample)))

True


In [145]:
import re

# ignore `1.`, ` 2.`, ` `, etc. and get the actual moves as separate entries
tokenizer_pattern = re.compile(r""" ?\d+\.|\. ?| ?[-\w#+]+|\s+""", flags=re.IGNORECASE)

print(re.findall(tokenizer_pattern, new_sample))

['1.', 'd4', ' d6', ' 2.', 'c4', ' g6', ' 3.', 'Nc3', ' Bg7', ' 4.', 'Nf3', ' Nf6', ' 5.', 'e3', ' O-O', ' 6.', 'Be2', ' Nbd7', ' 7.', 'O-O', ' c6', ' 8.', 'b4', ' e5', ' 9.', 'a4', ' Qe7', ' 10.', 'b5', ' Re8', ' 11.', 'a5', ' c5', ' 12.', 'dxe5', ' dxe5', ' 13.', 'e4', ' h6', ' 14.', 'Nd2', ' Nf8', ' 15.', 'Nd5', ' Qd6', ' 16.', 'Ba3', ' Ne6', ' 17.', 'Nb3', ' Qf8', ' 18.', 'f4', ' exf4', ' 19.', 'e5', ' Nd7', ' 20.', 'Bg4', ' Bxe5', ' 21.', 'Ra2', ' f5', ' 22.', 'Bf3', ' Qg7', ' 23.', 'Re1', ' g5', ' 24.', 'Bh5', ' Rf8', ' 25.', 'Rae2', ' Nd4', ' 26.', 'Rxe5', ' Nxe5', ' 27.', 'Nxd4', ' cxd4', ' 28.', 'Bxf8', ' Kxf8', ' 29.', 'Qe2', ' Bd7', ' 30.', 'Qxe5', ' Qxe5', ' 31.', 'Rxe5', ' Rc8', ' 32.', 'Be2', ' Be8', ' 33.', 'Bd3', ' Kg7', ' 34.', 'Re7+', ' Kf8', ' 35.', 'Rxb7', ' Bf7', ' 36.', 'a6', ' Re8', ' 37.', 'b6', ' axb6', ' 38.', 'a7', ' Bxd5', ' 39.', 'cxd5', ' Re1+', ' 40.', 'Kf2']
